In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

传输效率$\eta=P_2/P_1$，这里测定了50欧姆电阻上的电压，由于$P=U^2R$，故两个平方相除即可。

等效折射率$n=ct_p/L$，$\phi=-\omega t_p$，L为33.6m（或38.4m）

群速度$v_g=c/(n+\omega\frac{dn}{d\omega})$

In [2]:
dataC1 = pd.read_csv('dataC1.csv')
def dataCal(data,length,filename):
    #先重新计算phi，也可用np.unwarp
    sub360 = 0
    data['dphiF'] = data['dphi']
    for i in range(1,20):
        if (data['dphi'].iloc[i] > 0) & (data['dphi'].iloc[i-1] < 0):
            sub360 += 1
        data['dphiF'].iloc[i] -= sub360*360
    data['dphiF'] = data['dphiF'] * np.pi / 180
    eta = np.square(data['ch2']) / np.square(data['ch1'])
    tp = - data['dphiF'] / (np.arange(1,21)*1E6*2*np.pi)
    n = 299792458*tp/length
    vg=np.zeros(19)
    for i in range(1,20):
        vg[i-1] = 299792458/(n[i]+(i+1)*1E6*2*np.pi*((n[i]-n[i-1])/(1*1E6*2*np.pi)))
    dfEtaN = pd.DataFrame({'freq': range(1,21), 'eta': eta, 'n': n}).round(3)
    dfVg = pd.DataFrame({'freq': range(2,21), 'vg': vg / 1E8}).round(3)
    dfEtaN.to_csv(filename+'en.csv', index=False)
    dfVg.to_csv(filename+'vg.csv', index=False)
    return eta, n, vg
def plotData(eta, n, vg, filename):
    fig, axs = plt.subplots(2, 1, sharex=True, figsize=(8, 10))
    axs[0].set_ylim(0)
    axs[0].scatter(np.arange(1,21), eta)
    axs[0].plot(np.arange(1,21), eta, linestyle='--')
    axs[1].scatter(np.arange(1,21), n)
    axs[1].plot(np.arange(1,21), n, linestyle='--')
    axs[0].grid(True)
    axs[1].grid(True)
    axs[1].set_xlabel("freq(MHz)")
    axs[0].set_ylabel("$\eta$")
    axs[1].set_ylabel("index of refraction(n)")
    plt.savefig(filename+"-eta-n.pdf")
    plt.close()
    fig, ax = plt.subplots()
    ax.scatter(np.arange(2,21),vg)
    ax.plot(np.arange(2,21),vg,linestyle='--')
    ax.grid(True)
    ax.set_xlabel("freq(MHz)")
    ax.set_ylabel("$v_g$(m/s)")
    ax.axhline(y=299792458, color='lightcoral', linestyle='--')
    plt.savefig(filename+"-vg.pdf")
    plt.close()
eta, n, vg = dataCal(dataC1, 33.6, "C1")
plotData(eta, n, vg, "C1")
dataC4 = pd.read_csv("dataC4.csv")
eta, n, vg = dataCal(dataC4, 38.4, "C4")
plotData(eta, n, vg, "C4")

进行模拟：
一个矩阵：
$$
\begin{bmatrix}
	\cosh{\gamma l} & Z_{01}\sinh{\gamma l} \\
	\frac{1}{Z_{01}}\sinh{\gamma l} & \cosh{\gamma l}
\end{bmatrix}
$$

解：$v_L=\frac{e}{[1\quad Z_0]A_T[1\quad 1/Z_0]^T}$

计算群速度：$v_g=c/(n+\omega\frac{dn}{d\omega})$，其中$n=ct_p/L$，$\phi=-\omega t_p$

In [3]:
lc = 8.854187817E-12*2.354*1*1.25663706E-6
p0 = 0.25
freqa = np.arange(1,20.1,0.1)
vl = np.zeros(freqa.shape[0],dtype=np.complex128)
vld = np.zeros(freqa.shape[0],dtype=np.complex128)
i = 0
for freq in freqa:
    gamma = complex(1.810E-6,2*np.pi*freq*1E6*np.sqrt(lc))
    a1 = np.matrix([[np.cosh(gamma*4.8),25*np.sinh(gamma*4.8)],[1/25*np.sinh(gamma*4.8),np.cosh(gamma*4.8)]])
    a2 = np.matrix([[np.cosh(gamma*4.8),50*np.sinh(gamma*4.8)],[1/50*np.sinh(gamma*4.8),np.cosh(gamma*4.8)]])
    #无缺陷
    at = a1*a2*a1*a2*a1*a2*a1
    vl[i] = 10/(np.matrix([1,50])*at*np.matrix([[1],[1/50]]))
    #有缺陷
    atd = a1*a2*a1*a2*a2*a1*a2*a1
    vld[i] = 10/(np.matrix([1,50])*atd*np.matrix([[1],[1/50]]))
    i += 1
phi = np.angle(vl)
phi = np.unwrap(phi)
tp = - phi / (freqa*1E6*2*np.pi)
ns = 299792458*tp/33.6
vgs=np.zeros(freqa.shape[0]-1)
for i in range(1,freqa.shape[0]):
    vgs[i-1] = 299792458/(ns[i]+freqa[i]*1E6*2*np.pi*((ns[i]-ns[i-1])/((freqa[i]-freqa[i-1])*1E6*2*np.pi)))
phid = np.angle(vld)
phid = np.unwrap(phid)
tpd = - phid / (freqa*1E6*2*np.pi)
nd = 299792458*tpd/38.4
vgd=np.zeros(freqa.shape[0]-1)
for i in range(1,freqa.shape[0]):
    vgd[i-1] = 299792458/(nd[i]+freqa[i]*1E6*2*np.pi*((nd[i]-nd[i-1])/((freqa[i]-freqa[i-1])*1E6*2*np.pi)))
etas = np.square(np.abs(vl)) / 50
etad = np.square(np.abs(vld)) / 50

def plotDataSim(eta, n, vg, etas, ns, vgs, freqa,filename):
    fig, axs = plt.subplots(2, 1, sharex=True, figsize=(8, 10))
    axs[0].set_ylim(0)
    axs[0].scatter(np.arange(1,21), eta)
    axs[0].plot(freqa, etas)
    axs[1].scatter(np.arange(1,21), n)
    axs[1].plot(freqa, ns)
    axs[0].grid(True)
    axs[1].grid(True)
    axs[1].set_xlabel("freq(MHz)")
    axs[0].set_ylabel("$\eta$")
    axs[1].set_ylabel("index of refraction(n)")
    plt.savefig(filename+"-eta-n.pdf")
    plt.close()
    fig, ax = plt.subplots()
    ax.scatter(np.arange(2,21),vg)
    ax.plot(freqa[1:],vgs)
    ax.grid(True)
    ax.set_xlabel("freq(MHz)")
    ax.set_ylabel("$v_g$(m/s)")
    ax.axhline(y=299792458, color='lightcoral', linestyle='--')
    plt.savefig(filename+"-vg.pdf")
    plt.close()
eta, n, vg = dataCal(dataC1, 33.6, "C1")
plotDataSim(eta, n, vg, etas, ns, vgs, freqa, "normal")
eta, n, vg = dataCal(dataC4, 38.4, "C4")
plotDataSim(eta, n, vg, etad, nd, vgd, freqa, "defects")

/tmp/ipykernel_25309/2052796881.py:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  vl[i] = 10/(np.matrix([1,50])*at*np.matrix([[1],[1/50]]))
/tmp/ipykernel_25309/2052796881.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  vld[i] = 10/(np.matrix([1,50])*atd*np.matrix([[1],[1/50]]))
